### Import Libraries

In [1]:
import os
import numpy as np
import tensorflow as tf
import random
from PIL import Image
import matplotlib.pyplot as plt

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

### Set seed for reproducibility

In [3]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### Leaf Dataset

In [4]:
labels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']

### Data Preprocessing

Done in the other notebook

### Data Loader

In [5]:
sub_dataset_dir = '/kaggle/input/leaves/leaves/'
training_dir = os.path.join(sub_dataset_dir, 'training')
validation_dir = os.path.join(sub_dataset_dir, 'validation')
test_dir = os.path.join(sub_dataset_dir, 'testing')

In [6]:
# Images are divided into folders, one for each class. 
# If the images are organized in such a way, we can exploit the 
# ImageDataGenerator to read them from disk.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of ImageDataGenerator for training, validation, and test sets
# Create an instance of ImageDataGenerator with Data Augmentation
train_data_gen = ImageDataGenerator(#preprocessing_function=transformation,
                                    rotation_range=30,
                                    height_shift_range=50,
                                    width_shift_range=50,
                                    zoom_range=0.3,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    fill_mode='reflect', #)
                                    rescale=1/255.) # rescale value is multiplied to the image

valid_data_gen = ImageDataGenerator(rescale=1/255.)#,
                                    #preprocessing_function=transformation)

test_data_gen = ImageDataGenerator(rescale=1/255.)#,
                                    #preprocessing_function=transformation)


# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
train_gen = train_data_gen.flow_from_directory(directory=training_dir,
                                               target_size=(256,256),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=8,
                                               shuffle=True,
                                               seed=seed)

valid_gen = train_data_gen.flow_from_directory(directory=validation_dir,
                                               target_size=(256,256),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=8,
                                               shuffle=False,
                                               seed=seed)

test_gen = train_data_gen.flow_from_directory(directory=test_dir,
                                              target_size=(256,256),
                                              color_mode='rgb',
                                              classes=None, # can be set to labels
                                              class_mode='categorical',
                                              batch_size=8,
                                              shuffle=False,
                                              seed=seed)

### Model Metadata

In [7]:
# Model configuration
input_shape = (256, 256, 3)
epochs = 100
batch_size = 64
n_classes = 14
weight_decay = 1e-5
model_name = "CNN_gap_64_rgb"

### CNN Model

* Batch Norm
* Early Stopping
* Batch
* Increasing Dropout
* Weight Decay
* Weight Initialization
* Data Augmentation

In [8]:
# Model used for the exercise:
# (Conv + ReLU + MaxPool) x 4 + (Conv + ReLU + GlobalPooling) x 1 + FC x 2
def build_model(input_shape):

    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    conv1 = tfkl.Conv2D(
        filters=16,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        kernel_regularizer=tf.keras.regularizers.l2(weight_decay)
    )(input_layer)
    batch_norm1 = tfkl.BatchNormalization()(conv1)
    pool1 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(batch_norm1)

    conv2 = tfkl.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        kernel_regularizer=tf.keras.regularizers.l2(weight_decay)
    )(pool1)
    batch_norm2 = tfkl.BatchNormalization()(conv2)
    pool2 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(batch_norm2)

    conv3 = tfkl.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        kernel_regularizer=tf.keras.regularizers.l2(weight_decay)
    )(pool2)
    batch_norm3 = tfkl.BatchNormalization()(conv3)
    pool3 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(batch_norm3)

    conv4 = tfkl.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        kernel_regularizer=tf.keras.regularizers.l2(weight_decay)
    )(pool3)
    batch_norm4 = tfkl.BatchNormalization()(conv4)
    pool4 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(batch_norm4)

    conv5 = tfkl.Conv2D(
        filters=256,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        kernel_regularizer=tf.keras.regularizers.l2(weight_decay)
    )(pool4)
    batch_norm5 = tfkl.BatchNormalization()(conv5)
    globalPool = tfkl.GlobalAveragePooling2D()(batch_norm5)

    flattening_layer = tfkl.Flatten(name='Flatten')(globalPool)
    flattening_layer = tfkl.Dropout(0.25, seed=seed)(flattening_layer)
    classifier_layer = tfkl.Dense(units=512, name='Classifier', kernel_initializer=tfk.initializers.GlorotUniform(seed), activation='relu')(flattening_layer)
    batch_norm6 = tfkl.BatchNormalization()(classifier_layer)
    classifier_layer = tfkl.Dropout(0.5, seed=seed)(batch_norm6)
    output_layer = tfkl.Dense(units=n_classes, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='Output')(classifier_layer)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [9]:
# Build model
model = build_model(input_shape)
model.summary()

In [ ]:
#tfk.utils.plot_model(model)

### Training

In [10]:
# Utility function to create folders and callbacks for training
from datetime import datetime

def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('/kaggle/working/models')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                     save_weights_only=False, # True to save only weights
                                                     save_best_only=False) # True to save only the best epoch 
  callbacks.append(ckpt_callback)

  # Visualize Learning on Tensorboard
  # ---------------------------------
  #tb_dir = os.path.join(exp_dir, 'tb_logs')
  #if not os.path.exists(tb_dir):
  #    os.makedirs(tb_dir)
      
  # By default shows losses and metrics for both training and validation
  #tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir, 
  #                                             profile_batch=0,
  #                                             histogram_freq=1)  # if > 0 (epochs) shows weights histograms
  #callbacks.append(tb_callback)

  # Early Stopping
  # --------------
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
  callbacks.append(es_callback)

  return callbacks

In [11]:
# Create folders and callbacks and fit
callbacks = create_folders_and_callbacks(model_name=model_name)

# Train the model
history = model.fit(
    x = train_gen,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = valid_gen,
    callbacks=[callbacks]
).history

In [12]:
# Save best epoch model
model.save("/kaggle/working/models/" + model_name + "_best_" + str(datetime.now().strftime('%b%d_%H-%M-%S')))

### Testing

In [14]:
!zip -r /kaggle/working/models_r.zip /kaggle/working/models